In [10]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# 示例数据
input_dim = 35778
output_dim = 1
X = np.random.rand(100, input_dim).astype(np.float32)
y = np.random.rand(100, output_dim).astype(np.float32)

In [11]:
# 标准化数据
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# PCA降维
pca = PCA(n_components=100)  # 将维度减少到100
X_reduced = pca.fit_transform(X_scaled)

# 转换为Tensor
X_tensor = torch.tensor(X_reduced)
y_tensor = torch.tensor(y)

# 创建TensorDataset
dataset = TensorDataset(X_tensor, y_tensor)

# 创建DataLoader
batch_size = 16
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [12]:
# 定义简单的神经网络
class SimpleNN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, output_dim)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = nn.ReLU()(self.fc1(x))
        x = self.dropout(x)
        x = nn.ReLU()(self.fc2(x))
        x = self.fc3(x)
        return x

In [13]:
# 创建模型
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimpleNN(input_dim=100, output_dim=output_dim).to(device)

# 定义优化器和损失函数
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

# 训练模型
def train_model(train_loader, model, optimizer, criterion, device):
    model.train()
    total_loss = 0
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()\
        
        # 打印梯度
        print("Gradients:")
        for name, param in model.named_parameters():
            print(name, param.grad.mean())
            
    return total_loss / len(train_loader)

# 示例训练过程
for epoch in range(10):
    train_loss = train_model(train_loader, model, optimizer, criterion, device)
    print(f"Epoch {epoch}, train loss: {train_loss:.4f}")

Gradients:
fc1.weight tensor(-0.0071, device='cuda:0')
fc1.bias tensor(0.0121, device='cuda:0')
fc2.weight tensor(0.1672, device='cuda:0')
fc2.bias tensor(0.0389, device='cuda:0')
fc3.weight tensor(-4.0499, device='cuda:0')
fc3.bias tensor(-2.1630, device='cuda:0')
Gradients:
fc1.weight tensor(-0.0019, device='cuda:0')
fc1.bias tensor(0.0104, device='cuda:0')
fc2.weight tensor(0.1003, device='cuda:0')
fc2.bias tensor(0.0193, device='cuda:0')
fc3.weight tensor(-4.4128, device='cuda:0')
fc3.bias tensor(-1.5038, device='cuda:0')
Gradients:
fc1.weight tensor(0.0037, device='cuda:0')
fc1.bias tensor(0.0064, device='cuda:0')
fc2.weight tensor(0.1077, device='cuda:0')
fc2.bias tensor(0.0284, device='cuda:0')
fc3.weight tensor(-2.7543, device='cuda:0')
fc3.bias tensor(-1.4316, device='cuda:0')
Gradients:
fc1.weight tensor(-0.0039, device='cuda:0')
fc1.bias tensor(0.0087, device='cuda:0')
fc2.weight tensor(0.1333, device='cuda:0')
fc2.bias tensor(0.0259, device='cuda:0')
fc3.weight tensor(1.167

In [14]:
# 获取第一层全连接层的权重
fc1_weights = model.fc1.weight.data.cpu().numpy()

# 计算每个降维后特征的重要性
feature_importance_reduced = np.sum(np.abs(fc1_weights), axis=0)

In [15]:
# 获取PCA组件
components = pca.components_

# 计算每个原始特征的重要性
feature_importance_original = np.dot(feature_importance_reduced, np.abs(components))

# 对特征重要性进行排序
sorted_idx = np.argsort(feature_importance_original)[::-1]
sorted_importance = feature_importance_original[sorted_idx]

# 打印前10%个重要特征
top_10_percent = int(0.1 * len(sorted_importance))
for i in range(top_10_percent):
    print(f"Feature {sorted_idx[i]} importance: {sorted_importance[i]:.4f}")
top_10_percent

Feature 98 importance: 3.9472
Feature 88 importance: 2.0676
Feature 95 importance: 1.9779
Feature 94 importance: 1.9662
Feature 87 importance: 1.9529
Feature 93 importance: 1.8165
Feature 11 importance: 1.6871
Feature 97 importance: 1.6370
Feature 30 importance: 1.6269
Feature 89 importance: 1.6211
Feature 81 importance: 1.5698
Feature 16 importance: 1.5595
Feature 91 importance: 1.5427
Feature 8 importance: 1.5397
Feature 26 importance: 1.5383
Feature 96 importance: 1.5363
Feature 92 importance: 1.5351
Feature 18 importance: 1.5288
Feature 14 importance: 1.5236
Feature 36 importance: 1.5216
Feature 60 importance: 1.5063
Feature 28 importance: 1.5063
Feature 86 importance: 1.4950
Feature 41 importance: 1.4874
Feature 25 importance: 1.4865
Feature 24 importance: 1.4834
Feature 38 importance: 1.4793
Feature 64 importance: 1.4729
Feature 80 importance: 1.4707
Feature 13 importance: 1.4688
Feature 4 importance: 1.4649
Feature 37 importance: 1.4649
Feature 71 importance: 1.4624
Feature 15 i

3577